In [1]:
import numpy as np
import pandas as pd

In [2]:
wiki = pd.read_csv('paths_finished.csv')
wiki['path_list'] = wiki['path'].apply(lambda x: [[y] for y in x.split(';')])
wiki.head()

,hashedIpAddress,timestamp,durationInSec,path,rating,path_list
0,6a3701d319fc3754,1297740409,166,14th_century;15th_century;16th_century;Pacific...,NaN,"[[14th_century], [15th_century], [16th_century..."
1,3824310e536af032,1344753412,88,14th_century;Europe;Africa;Atlantic_slave_trad...,3.0,"[[14th_century], [Europe], [Africa], [Atlantic..."
2,415612e93584d30e,1349298640,138,14th_century;Niger;Nigeria;British_Empire;Slav...,NaN,"[[14th_century], [Niger], [Nigeria], [British_..."
3,64dd5cd342e3780c,1265613925,37,14th_century;Renaissance;Ancient_Greece;Greece,NaN,"[[14th_century], [Renaissance], [Ancient_Greec..."
4,015245d773376aab,1366730828,175,14th_century;Italy;Roman_Catholic_Church;HIV;R...,3.0,"[[14th_century], [Italy], [Roman_Catholic_Chur..."


In [3]:
path = wiki['path_list']
freq1= set()
[[[freq1.add(x) for x in item] for item in row] for row in path]
freq1 = list(freq1)
path

0        [[14th_century], [15th_century], [16th_century...
1        [[14th_century], [Europe], [Africa], [Atlantic...
2        [[14th_century], [Niger], [Nigeria], [British_...
3        [[14th_century], [Renaissance], [Ancient_Greec...
4        [[14th_century], [Italy], [Roman_Catholic_Chur...
                               ...                        
51313           [[Yagan], [Ancient_Egypt], [Civilization]]
51314    [[Yagan], [Folklore], [Brothers_Grimm], [<], [...
51315    [[Yagan], [Australia], [England], [France], [U...
51316    [[Yarralumla%2C_Australian_Capital_Territory],...
51317                    [[Ziad_Jarrah], [Germany], [Jew]]
Name: path_list, Length: 51318, dtype: object

### converting list of strings to ints so less memory is used

In [8]:
lists = wiki['path'].str.split(';')
id_lists = []
for row in lists:
    for item in row:
        list_row = [[freq1.index(item)] for item in row]
    id_lists.append(list_row)
id_lists


[[[3026], [2483], [4067], [3466], [2879], [1189], [1046], [1586], [2995]],
 [[3026], [436], [1046], [1586], [2995]],
 [[3026], [4027], [3205], [2654], [3578], [1046], [1586], [2995]],
 [[3026], [1596], [1556], [3082]],
 [[3026], [1585], [2919], [1149], [1204], [1296], [1977]],
 [[3026], [436], [3520], [2023], [1296], [1977]],
 [[3026], [1234], [3070], [780]],
 [[3026], [4031], [3748], [141], [2174], [1021]],
 [[3026], [4031], [141], [1021]],
 [[3026], [2483], [1896], [2714], [526], [2174], [1021]],
 [[3026],
  [4031],
  [1391],
  [2714],
  [3597],
  [3080],
  [2882],
  [2541],
  [1437],
  [526],
  [2174],
  [141],
  [1021]],
 [[3026], [3192], [846], [2355], [2714], [1437], [526], [2098], [141], [1021]],
 [[3026], [4031], [2761], [141], [1021]],
 [[3026],
  [1596],
  [3887],
  [2714],
  [3597],
  [3080],
  [2882],
  [2063],
  [2882],
  [3080],
  [1437],
  [2882],
  [2882],
  [2541],
  [3748],
  [1021]],
 [[3026], [1596], [3902], [3880], [2063], [1314], [2882], [2882], [1021]],
 [[3026],

In [310]:

visited_itemsets = set()

# we create an dictionary for items,occurrance for each itemset > MIN_SUP
# then loop over the items and make recursive calls for each item creating a sub-dataset with new projection
# return pattern list containing "alphas"
patterns = []
def prefixSpan(alpha,dataset,min_sup):
    
    itemset = {}
    if not dataset:
            return alpha
        
    for seq in dataset:
        unique = []
        for item in seq:
            for it in item:
                if it not in unique:
                    unique.append(it)
                    itemset[it] = itemset.get(it,0) + 1  
            if len(item) > 1:
                it = "".join(item)
                if it not in unique:
                    unique.append(it)
                    itemset[it] = itemset.get(it,0) + 1  
                
    freq_itemset = {key:value for key,value in itemset.items() if value >= min_sup}

    new_data = []
    
    def add_new_k_item(seq):
        projected_seq = seq
        hashed_projected = str(projected_seq)
        if hashed_projected not in visited_itemsets:
            visited_itemsets.add(hashed_projected)
            new_data.append(projected_seq)
            
    for freq_item,freq_occurr in freq_itemset.items():
        alpha_dashed = f"{alpha}{freq1[freq_item]},"
        
            
        for seq in dataset:
            found = False
            for i,k_itemset in enumerate(seq):
                if found:
                    continue
                if freq_item in k_itemset:
                    found = True
                    if len(k_itemset) > 1 and freq_item != k_itemset[-1]: # k_item has multi items and not the last one
                        j = k_itemset.index(freq_item)
                        add_new_k_item([k_itemset[j+1:],*seq[i+1:]])
                        continue
                        
                    if k_itemset == seq[-1] and i == (len(seq) - 1): # k_item has one item and in the last
                        continue
                    
                    else:
                        add_new_k_item([*seq[i+1:]]) # k_item has one item
                        continue
        if alpha_dashed not in patterns:
            patterns.append(f"{alpha_dashed}: {freq_occurr}")
            prefixSpan(alpha_dashed,new_data,min_sup)
    
    return patterns

![](imgs/resultsarecoming.jpeg "BRACE YOURSELVES")

In [315]:
# dataset has long sequences, it's better to analyze it and remove long sequences, anyway I tested a part of it
visited_itemsets = set()
patterns = []
prefixSpan("",id_lists[:4000],min_sup=10)

['14th_century,: 49',
 '14th_century,Time,: 10',
 '14th_century,Time,Rainbow,: 10',
 '14th_century,Color,: 10',
 '14th_century,Color,Rainbow,: 11',
 '14th_century,Rainbow,: 22',
 '14th_century,Rainbow,Rainbow,: 11',
 '15th_century,: 12',
 '15th_century,Time,: 10',
 '15th_century,Color,: 11',
 '15th_century,Rainbow,: 23',
 '16th_century,: 12',
 '16th_century,Time,: 10',
 '16th_century,Color,: 11',
 '16th_century,Rainbow,: 23',
 'Pacific_Ocean,: 39',
 'Pacific_Ocean,Time,: 10',
 'Pacific_Ocean,Color,: 11',
 'Pacific_Ocean,Rainbow,: 23',
 'Pacific_Ocean,<,: 15',
 'Pacific_Ocean,<,<,: 10',
 'Atlantic_Ocean,: 119',
 'Atlantic_Ocean,Africa,: 12',
 'Atlantic_Ocean,United_States,: 17',
 'Atlantic_Ocean,Time,: 10',
 'Atlantic_Ocean,Color,: 11',
 'Atlantic_Ocean,Rainbow,: 23',
 'Atlantic_Ocean,<,: 26',
 'Atlantic_Ocean,Ocean,: 14',
 'Atlantic_Ocean,Shark,: 12',
 'Atlantic_Ocean,Earth,: 11',
 'Atlantic_Ocean,Fish,: 13',
 'Atlantic_Ocean,Fish,<,: 11',
 'Africa,: 214',
 'Africa,Africa,: 13',
 'Afri

In [316]:
len(patterns)

257724

### Top frequent k_items

In [317]:
sorted(patterns,key=lambda x : x.split(':')[1],reverse=True)[:10]

['Caribbean_Sea,<,: 998',
 'Middle_Ages,<,: 998',
 'Desert,<,: 998',
 'Rome,<,: 997',
 'Aircraft,<,: 995',
 'United_States,United_States,: 99',
 '<,World_War_II,<,: 99',
 'Biology,Computer,: 99',
 'Art,Fruit,: 99',
 'Fishing,Fruit,: 99']

###### test case

In [309]:
visited_itemsets = set()
patterns = []
prefixSpan(alpha="",dataset=[
[["a"], ["a", "b", "c"], ["a", "c"], ["c"]],
[["a"], ["c"], ["b", "c"]],
[["a", "b"], ["d"], ["c"], ["b"], ["c"]],
[["a"], ["c"], ["b"], ["c"]]
],min_sup=2)

['a,: 4',
 'a,b,: 4',
 'a,b,c,: 3',
 'a,b,c,c,: 2',
 'a,c,: 4',
 'a,c,c,: 4',
 'a,c,b,: 2',
 'b,: 4',
 'b,b,: 4',
 'b,c,: 4',
 'c,: 4',
 'c,b,: 4',
 'c,c,: 4']